# Setup

In [1]:
from datetime import datetime, timezone
import numpy as np
import pandas as pd

# Data Load

In [2]:
data_path = "Data/"

Set Data Types for Efficient Memory Usage

In [3]:
attr_types = {
   'app_id': 'category',
   'session_id': 'category',
   'attribute': np.int16,
   'attribute_value': 'category',
   'user_id_hash': 'category'
}

event_types = {
   'user_id_hash': 'category',
   'app_id': 'category',
   'session_id': 'category',
   'event': 'category',
   'event_timestamp': np.int64,
   'event_value': np.float16
}

sessions_types = {
   'app_id': 'category',
   'session_id': 'category',
   'start_timestamp': np.int64,
   'timezone': 'category',
   'timezone_offset': np.float64,
   'previous_sessions_duration': np.int64,
   'user_created_timestamp': np.int64,
   'is_user_first_session': bool,
   'is_session': bool,
   'is_wau': bool,
   'is_mau': bool,
   'country': 'category',
   'region': 'category',
   'city': 'category',
   'latitude': np.float16,
   'longitude': np.float16,
   'locale': 'category',
   'os_name': 'category',
   'session_index': np.int32,
   'device_id': 'category',
   'user_id_hash': 'category'
}

In [4]:
sessions = pd.read_csv(data_path + 'sessions.csv', dtype=sessions_types)
events = pd.read_csv(data_path + 'events.csv', dtype=event_types)

# Filtering
Read in sessions and events from sessions, and remove sessions that are not true sessions:

In [5]:
sessions = sessions[(sessions.is_session == True) & (sessions.is_wau == False) & (sessions.is_mau == False) & (sessions.is_developer == False)]

# Compute Purchase Targets

In [6]:
def get_purchasing_users(start_date, end_date):
    '''Identifies users who purchased between start_date and end_date'''
    pattern = '%d.%m.%Y %H:%M:%S'
    label_start_date = int(datetime.strptime(start_date, pattern).replace(tzinfo=timezone.utc).timestamp())*1000
    label_end_date = int(datetime.strptime(end_date, pattern).replace(tzinfo=timezone.utc).timestamp())*1000
    purchasing_users = list(events[(events.event_timestamp > label_start_date) & (events.event_timestamp < label_end_date) & (events.event == '8')]['user_id_hash'].unique())
    return purchasing_users

Find users who made purchases between Dec 1st and Dec 7th (7 day) and Dec 1st and Dec 14th (14 day)

In [7]:
start_date_14 = '01.12.2018 00:00:00'
end_date_14 = '14.12.2018 23:59:59'
start_date_7 = '01.12.2018 00:00:00'
end_date_7 = '07.12.2018 23:59:59'

purchasing_users_7day = get_purchasing_users(start_date_7, end_date_7)
purchasing_users_14day = get_purchasing_users(start_date_14, end_date_14)

Create a dataframe containing all users in the sessions table, identifying whether they purchased within the periods of interest

In [8]:
label_df = pd.DataFrame({'user_id_hash': list(sessions['user_id_hash'].unique())})
label_df["purchase_7day"] = np.where(label_df["user_id_hash"].isin(purchasing_users_7day), 1, 0)
label_df["purchase_14day"] = np.where(label_df["user_id_hash"].isin(purchasing_users_14day), 1, 0)

Shuffle labels and split dataframe to get training and validation labels:

In [9]:
label_df = label_df.sample(frac=1).reset_index(drop=True)
n = len(label_df)
train_labels_df = label_df.iloc[:int(0.8*n), ]
val_labels_df = label_df.iloc[int(0.8*n):, ]

Showing that the proportions of purchasers are not too different between the train/val/overall sets:

In [10]:
print("Overall Data:")
print(f"\t7-Day Purchase Proportion to Total: {label_df.purchase_7day.value_counts()[1]/len(label_df)}\n"
      f"\t14-Day Purchase Proportion to Total: {label_df.purchase_14day.value_counts()[1]/len(label_df)}\n")

print("Training Data:")
print(f"\t7-Day Purchase Proportion to Total: {train_labels_df.purchase_7day.value_counts()[1]/len(train_labels_df)}\n"
      f"\t14-Day Purchase Proportion to Total: {train_labels_df.purchase_14day.value_counts()[1]/len(train_labels_df)}\n")

print("Validation Data:")
print(f"\t7-Day Purchase Proportion to Total: {val_labels_df.purchase_7day.value_counts()[1]/len(val_labels_df)}\n"
      f"\t14-Day Purchase Proportion to Total: {val_labels_df.purchase_14day.value_counts()[1]/len(val_labels_df)}")

Overall Data:
	7-Day Purchase Proportion to Total: 0.0076333413503056405
	14-Day Purchase Proportion to Total: 0.009888316581089523

Training Data:
	7-Day Purchase Proportion to Total: 0.007681365576102418
	14-Day Purchase Proportion to Total: 0.009925042623810821

Validation Data:
	7-Day Purchase Proportion to Total: 0.007441244834710744
	14-Day Purchase Proportion to Total: 0.009741412706611571


Feather Dataframes to easily work with later

In [11]:
label_df.reset_index(drop=True, inplace=True)
train_labels_df.reset_index(drop=True, inplace=True)
val_labels_df.reset_index(drop=True, inplace=True)

In [12]:
label_df.to_feather('label_df')
train_labels_df.to_feather('train_labels_df')
val_labels_df.to_feather('val_labels_df')

In [13]:
label_df.head()

,user_id_hash,purchase_7day,purchase_14day
0,f2d439465878d7dbc0fd36113f4881c3251dec3e2013e3...,0,0
1,7de859dda06231b951984d592b8ffcd662f901f6b81338...,0,0
2,ec42dbdaa0ad7dfb7c6bb97943bf3c62064351283e97a5...,0,0
3,2ce42224f40bf9787f127f588bc59401de6cb3c9787f59...,0,0
4,093907bc060334e4106fd6d7eac8dbe4a956b31b44b67a...,0,0


In [14]:
train_labels_df.head()

,user_id_hash,purchase_7day,purchase_14day
0,f2d439465878d7dbc0fd36113f4881c3251dec3e2013e3...,0,0
1,7de859dda06231b951984d592b8ffcd662f901f6b81338...,0,0
2,ec42dbdaa0ad7dfb7c6bb97943bf3c62064351283e97a5...,0,0
3,2ce42224f40bf9787f127f588bc59401de6cb3c9787f59...,0,0
4,093907bc060334e4106fd6d7eac8dbe4a956b31b44b67a...,0,0


In [15]:
val_labels_df.head()

,user_id_hash,purchase_7day,purchase_14day
0,a16ebb025ffdcd59abf37a871b41a46ce602537c0699c3...,0,0
1,267d80cc59de3240e9590788a35b791f6828d16df6f56e...,0,0
2,a4e3ebe4316ae23de6b50fdd4757198da4220b6ab70118...,0,0
3,bd604ca5ee0d3648c9940d87ed2ad6934df25080f3c9e0...,0,0
4,2f60b9fbb7dcc167130c604f6a53dabb970da19466e2ce...,0,0
